# Homework 4

## Question 1

In [3]:
import pandas as pd

In [4]:
db = pd.read_csv('lalonde.csv')


db.head()

,id,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899


## Question 2

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

#DON'T DO IT TO OFTEN, IT TAKES SEVERAL MINUTES !
newsgroups_train = fetch_20newsgroups(data_home='.',subset='train',download_if_missing=True)

In [6]:
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [7]:
newsgroups_train['filenames']

array(['.\\20news_home\\20news-bydate-train\\rec.autos\\102994',
       '.\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51861',
       '.\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51879',
       ...,
       '.\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60695',
       '.\\20news_home\\20news-bydate-train\\comp.graphics\\38319',
       '.\\20news_home\\20news-bydate-train\\rec.motorcycles\\104440'],
      dtype='<U64')

In [8]:
newsgroups_train['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']